In [25]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

## The panelists by country

Reading in the panel information Netquest gave us.

### may need to be set

In [26]:
paneldate = "191120"

### needs oversight

In [27]:
outdir = "../out/panel_country/"

In [28]:
paneldir = ("C:/Users/schadem/Box Sync/LAPOP Shared/working documents/maita/Coordination/"+
            "IDB Online Trust/raw/netquest/"+paneldate+"/")
panelfile = [s for s in os.listdir(paneldir) if s.endswith(".csv")][0]
print("loading panel data: ", panelfile)


loading panel data:  VANDUS 1147438_W4_20191120.csv


### this can just do its thing

In [29]:
panels = pd.read_csv(paneldir+panelfile,
                     sep=';', 
#                      skiprows=2891450,
#                      nrows=1000000,
                     na_values=[' ','.'], 
                     encoding='latin1',
                     #error_bad_lines=False
                    )

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (536,537) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
[k for k in panels.columns if "departamento" in k]
[k for k in panels.columns if "COUNTRY" in k]

['COUNTRY']

Creating a dictionary of individual country dataframes, and cleaning them up

In [31]:
panels_dict = {}
for p in panels.COUNTRY.unique():
    panels_dict[p] = panels[panels.COUNTRY==p]#.dropna(axis=1,how='all')

In [32]:
panels_dict.keys()

dict_keys(['BR', 'MX', 'AR', 'PE', 'CL', 'CO'])

Where does the data go?

In [33]:
for pais, data in panels_dict.items():
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    data.to_csv(outdir+pais+"_netquest-panel.csv", encoding='utf8',index=False)

In [34]:
panels_dict["PE"]["PE_NSE_comodities#7"].value_counts()

1.0    61698
0.0    54486
Name: PE_NSE_comodities#7, dtype: int64

Next, bring the variables and labels in order.

In [35]:
manual_dict = pd.read_excel(paneldir+"Manual.xlsx",skiprows=1,sheet_name=None,)
variables = manual_dict['Variables']

In [36]:
levels = pd.concat(
    [manual_dict[k] for k in manual_dict.keys() if "Código" in k]
).fillna(method='ffill')
levels.columns = ["Variable","Valor","Etiqueta"]
#levels = pd.read_excel(paneldir+"manual_levels.xlsx")
# this expects a the first and second sheet of netquest background export manual
# with first line (title) removed

In [37]:
print(("Variable" in variables.columns) & ("Variable" in levels.columns))

True


In [38]:
variables_dict = {}
levels_dict = {}
countryexp = re.compile("^([A-Z]{2,2})_")

for pais in panels_dict:
    all_countries = variables.Variable.apply(countryexp.findall).apply(lambda l: l[0].lower() if len(l)>0 else "None").unique()
    other_countries = [p.lower() for p in all_countries if not p==pais]
    var_df = variables[(variables.Variable.str.lower().str.startswith(pais.lower())) #country-specific var
                       | (variables.Variable.apply(lambda s: sum([s.lower().startswith(k) for k in all_countries]))==0) #not specific for another country
                       | variables.Variable.isin(["panelistAge","DESKTOP_RESPONDENT","DEVICE","DESK","TARGET",
                                                  "SOCIODEMOGRAPHICS_DATE_V0","SOCIODEMOGRAPHICS_QUEST_V0"])
                       | variables.Variable.str.contains("PNuevo")
                      ]
                       
    lev_df = levels[levels.Variable.isin(var_df.Variable)]
    
    variables_dict[pais] = var_df
    levels_dict[pais] = lev_df
    
    var_df.to_excel(outdir+pais+'_variables.xlsx',index=False)
    lev_df.to_excel(outdir+pais+'_levels.xlsx',index=False)
    

## needs supervision
Check which variables are dropped at this stage

In [65]:
list(filter(lambda x: sum([x in v[1].Variable.unique() for v in variables_dict.items()])==0, variables.Variable))

['BO_departamento',
 'BO_education_level',
 'BO_education_level_hhousehold',
 'BO_laboral_situation',
 'BO_laboral_situation_hhousehold',
 'BO_municipio',
 'BO_provincia',
 'CRCSOCIAL',
 'CR_canton',
 'CR_cp',
 'CR_distrito',
 'CR_education_level',
 'CR_education_level_hhousehold',
 'CR_laboral_situation',
 'CR_laboral_situation_hhousehold',
 'CR_NSE_cars',
 'CR_NSE_computer',
 'CR_NSE_flights',
 'CR_NSE_internet',
 'CR_NSE_lights',
 'CR_NSE_maid',
 'CR_NSE_rooms',
 'CR_NSE_washingmachine',
 'CR_NSE_year_car',
 'CR_occupation',
 'CR_occupation_hhousehold',
 'CR_provincia',
 'DO_cp',
 'DO_distrito_municipal',
 'DO_education_level',
 'DO_education_level_hhousehold',
 'DO_laboral_situation',
 'DO_laboral_situation_hhousehold',
 'DO_municipio',
 'DO_provincia',
 'ECCSOCIAL',
 'EC_canton',
 'EC_cp',
 'EC_education_level',
 'EC_education_level_hhousehold',
 'EC_laboral_situation',
 'EC_laboral_situation_hhousehold',
 'EC_NSE_autos',
 'EC_NSE_baths',
 'EC_NSE_book',
 'EC_NSE_cellphone',
 'EC_